<center>

<font size="5">MGMTMSA 403: Optimization</font>

</center>


<center>

<font size="5">Assignment 3: Predicting Airbnb Prices</font>

</center>


<div style="display: flex; justify-content: space-between;">
    <div style="text-align: left;">
        Due on BruinLearn by 11:59pm on February 7th.
    </div>
    <div style="text-align: right;">
        Arnav Garg (906310841) <br>
        Oscar Trumpy (706310371) <br>
        Facundo Klappenbach (306316186) <br>
    </div>
</div>


**<font size="5">Background</font>**

The file AirbnbTrain.csv contains data on 1700 Airbnb listings in Hollywood, CA. The dataset contains features such as the location (by latitude and longitude), number of guests accomodated, number of beds, and other variables. The dataset also contains the price per night of each Airbnb listing. Your assignment will be to formulate an optimization model to predict the price of Airbnb listings using this dataset.
<br>
<br>
Note: Given a set of model coefficients β1, β2, . . . , βd, the average prediction error of the model β for a data set (xi,yi), i = 1,...,n is given by

\begin{align}
{Error} = \frac{1}{n} \sum_{i=1}^n | y_i - \sum_{j=1}^d \beta_j x_{ij} |
\end{align}

**<font size="5">Questions</font>**

**Model 1.** Formulate the least absolute deviations regression problem as a linear program. Solve the linear program using the data given in the file AirbnbTrain.csv. What is the prediction error, in $/night, of your model on the test set (provided in AirbnbTest.csv)?


The solution can be given by the following equations:

\begin{align}
\text{minimize} \;\; & \frac{1}{m} \sum_{i=1}^m Z_i \\
\text{s.t.} \;\; & Z_i \ge y_i - (\sum_{j=1}^n β_j \times X_{ij}) \quad i = 1 \ldots m\\
            & Z_i \ge (\sum_{j=1}^n β_j \times X_{ij}) - y_i \quad i = 1 \ldots m \\
\end{align}

In [32]:
# Import libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from gurobipy import *

# Load dataset
train = pd.read_csv('AirbnbTrain.csv')
test = pd.read_csv('AirbnbTest.csv')

# Display the first 5 rows of the dataset
train.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1,55,1,1,80,0,85
3,34.101028,-118.317848,0,2,1.0,1,1,20,1,8,98,0,75
4,34.098292,-118.324980,1,2,1.0,1,1,20,1,11,96,0,130


In [33]:
y = train['price']
m = train.shape[0]
n = train.drop(columns=['price']).shape[1]
x = train.drop(columns=['price'])

###############################################
# Initialize the model 
mod = Model()

###############################################
# Define Variables 
z = mod.addVars(m)
b = mod.addVars(n)

###############################################
# Define Constraints
for i in range(m):
    mod.addConstr(z[i] >= y[i] - sum(b[j] * x.iloc[i, j] for j in range(n - 1)))

for i in range(m):
    mod.addConstr(z[i] >= sum(b[j] * x.iloc[i, j] for j in range(n - 1)) - y[i])
    
###############################################
# Construct Objective
mod.setObjective((sum(z[i] for i in range(m)) * 1/m ), GRB.MINIMIZE)

###############################################
# Update and solve
mod.update()
mod.optimize()

###############################################
# Print Optimal Solution
print("Optimal Value:", mod.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3400 rows, 1712 columns and 39312 nonzeros
Model fingerprint: 0xefadf137
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 3400 rows, 1711 columns, 39312 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 11
 AA' NZ     : 2.843e+04
 Factor NZ  : 3.107e+04 (roughly 2 MB of memory)
 Factor Ops : 3.757e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13504409e+03 

In [34]:
###############################################
# Print and Calculate the Error
test['price_pred'] = 0
for j in range(n - 1):
    test['price_pred'] += test.iloc[:, j] * b[j].x
    test['price_pred'] = test['price_pred'] + b[n - 1].x

print("Mean Absolute Error:", mean_absolute_error(test['price'], test['price_pred']))

Mean Absolute Error: 35.64907272288845


The prediction error for the test set of our model is ~$35.65 per night.

**Model 2.** Suppose that to improve interpretability, you wish to build a model that predicts Airbnb prices using only the three most important variables. Modify **Model 1** by including a constraint that allows at most three variables to have non-zero coefficients.


The solution can be given by the following equations:

\begin{align}
\text{minimize} \;\; & \frac{1}{m} \sum_{i=1}^m Z_i \\
\text{s.t.} \;\; & Z_i \ge y_i - (\sum_{j=1}^n β_j \times X_{ij}) \quad i = 1 \ldots m\\
            & Z_i \ge (\sum_{j=1}^n β_j \times X_{ij}) - y_i \quad i = 1 \ldots m \\
            & \sum_{j=1}^n F_j \le 3 \\
            & β_j \le F_j \times 1000 \quad j = 1 \ldots n\\
            & β_j \ge F_j \times (-1000) \quad j = 1 \ldots n\\
            
            
\end{align}

In [35]:
y = train['price']
m = train.shape[0]
n = train.drop(columns=['price']).shape[1]
x = train.drop(columns=['price'])

###############################################
# Initialize the model 
mod = Model()

###############################################
# Define Variables 
z = mod.addVars(m)
b = mod.addVars(n)
f = mod.addVars(n, vtype = GRB.BINARY)

###############################################
# Define Constraints
for i in range(m):
    mod.addConstr(z[i] >= y[i] - sum(b[j] * x.iloc[i, j] for j in range(n - 1)))

for i in range(m):
    mod.addConstr(z[i] >= sum(b[j] * x.iloc[i, j] for j in range(n - 1)) - y[i])
 
mod.addConstr(sum(f[j] for j in range(n)) <= 3) 

for j in range(n):
    mod.addConstr(b[j] <= f[j] * 1000)

for j in range(n):
    mod.addConstr(b[j] >= f[j] * -1000)

###############################################
# Construct Objective
mod.setObjective((sum(z[i] for i in range(m)) * 1/m ), GRB.MINIMIZE)

###############################################
# Update and solve
mod.update()
mod.optimize()

###############################################
# Print Optimal Solution
print("Optimal Value:", mod.objVal)
for j in range(n):
    print(f'b{j} {b[j].x}')

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3425 rows, 1724 columns and 39372 nonzeros
Model fingerprint: 0xde9bf761
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 841 rows and 416 columns
Presolve time: 0.02s
Presolved: 2584 rows, 1308 columns, 29751 nonzeros
Variable types: 1297 continuous, 11 integer (11 binary)

Root relaxation: objective 3.649240e+01, 1416 iterations, 0.11 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.49240    0    6  144.96824   

a) List the names and coefficients of the three variables selected by the optimization model.

In [36]:
# List the names and coefficients of the three variables selected by the optimization model.
print("The three variables selected by the optimization model are:")
for j in range(n):
    if f[j].x == 1:
        print(f'Variable {j} is "{train.columns[j]}" with coefficient {b[j].x}')

The three variables selected by the optimization model are:
Variable 2 is "Entire home" with coefficient 52.0
Variable 3 is "accommodates" with coefficient 14.0
Variable 5 is "bedrooms" with coefficient 32.0


b) What is the new prediction error, in $/night, of **Model 2**?

In [37]:
# Print and Calculate the Error
test['price_pred'] = 0
for j in range(n - 1):
    test['price_pred'] += test.iloc[:, j] * b[j].x
    test['price_pred'] = test['price_pred'] + b[n - 1].x

print("Mean Absolute Error:", mean_absolute_error(test['price'], test['price_pred']))

Mean Absolute Error: 37.73676680972818


The prediction error for the test set of our model is ~$37.73 per night.

**Model 3.** Suppose now you wish to build a model that predicts Airbnb listing price using only three variables, where one of the variables is the number of beds.


The solution can be given by the following equations:

\begin{align}
\text{minimize} \;\; & \frac{1}{m} \sum_{i=1}^m Z_i \\
\text{s.t.} \;\; & Z_i \ge y_i - (\sum_{j=1}^n β_j \times X_{ij}) \quad i = 1 \ldots m\\
            & Z_i \ge (\sum_{j=1}^n β_j \times X_{ij}) - y_i \quad i = 1 \ldots m \\
            & \sum_{j=1}^n F_j \le 3 \\
            & β_j \le F_j \times 1000 \quad j = 1 \ldots n\\
            & β_j \ge F_j \times (-1000) \quad j = 1 \ldots n\\
            & F_6 = 1 \\
\end{align}

In [38]:
y = train['price']
m = train.shape[0]
n = train.drop(columns=['price']).shape[1]
x = train.drop(columns=['price'])

###############################################
# Initialize the model 
mod = Model()

###############################################
# Define Variables 
z = mod.addVars(m)
b = mod.addVars(n)
f = mod.addVars(n, vtype = GRB.BINARY)

###############################################
# Define Constraints
for i in range(m):
    mod.addConstr(z[i] >= y[i] - sum(b[j] * x.iloc[i, j] for j in range(n - 1)))

for i in range(m):
    mod.addConstr(z[i] >= sum(b[j] * x.iloc[i, j] for j in range(n - 1)) - y[i])
 
mod.addConstr(sum(f[j] for j in range(n)) <= 3) 

for j in range(n):
    mod.addConstr(b[j] <= f[j] * 1000)

for j in range(n):
    mod.addConstr(b[j] >= f[j] * -1000)

mod.addConstr(f[6] == 1)

###############################################
# Construct Objective
mod.setObjective((sum(z[i] for i in range(m)) * 1/m ), GRB.MINIMIZE)

###############################################
# Update and solve
mod.update()
mod.optimize()

###############################################
# Print Optimal Solution
print("Optimal Value:", mod.objVal)
for j in range(n):
    print(f'b{j} {b[j].x}')

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3426 rows, 1724 columns and 39373 nonzeros
Model fingerprint: 0x7002e2af
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 843 rows and 417 columns
Presolve time: 0.02s
Presolved: 2583 rows, 1307 columns, 29748 nonzeros
Variable types: 1297 continuous, 10 integer (10 binary)

Root relaxation: objective 3.649240e+01, 1416 iterations, 0.11 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.49240    0    7  144.96824   

a) List the names and coefficients of the two other variables selected by the optimization model.

In [39]:
# List the names and coefficients of the three variables selected by the optimization model.
print("The three variables selected by the optimization model are:")
for j in range(n):
    if f[j].x == 1:
        print(f'Variable {j} is "{train.columns[j]}" with coefficient {b[j].x}')

The three variables selected by the optimization model are:
Variable 2 is "Entire home" with coefficient 67.875
Variable 5 is "bedrooms" with coefficient 47.375
Variable 6 is "beds" with coefficient 12.125


b) Which variable was in **Model 2** but is no longer in **Model 3**? Briefly explain in 1-2 sentences why this variable might have been dropped.

Variable 3 which is "accomodates" was dropped from the model because it has the lowest predictive power out of the three choices between "entire home", "accomodates" and "bedrooms". Therefore "bedrooms" and "entire home" is the best combination of two variables that along with "beds" minimise the mean absolute error.

c) What is the new prediction error, in $/night, of **Model 3**?

In [40]:
# Print and Calculate the Error
test['price_pred'] = 0
for j in range(n - 1):
    test['price_pred'] += test.iloc[:, j] * b[j].x
    test['price_pred'] = test['price_pred'] + b[n - 1].x

print("Mean Absolute Error:", mean_absolute_error(test['price'], test['price_pred']))

Mean Absolute Error: 38.59960658082976


The prediction error for the test set of our model is ~$38.60 per night.